<a href="https://colab.research.google.com/github/aditivin16/AI_Club_RLGames/blob/main/aditi_a2c_2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install swig
!pip3 install gym[box2d]
import gym
import torch
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
actor_lr = 0.01
critic_lr = 0.01
gamma = 0.99
num_episodes = 100


# Define the actor and critic networks
actor = nn.Sequential(nn.Linear(8, 64),nn.ReLU(),nn.Linear(64, 64),nn.ReLU(),nn.Linear(64, 4),nn.Softmax(dim=-1)) #softmax to convert to action probabilities
critic = nn.Sequential(nn.Linear(8, 64),nn.ReLU(),nn.Linear(64, 64),nn.ReLU(),nn.Linear(64, 1))

# Defining the optimizer for actor and critic
optimizer_actor = optim.Adam(actor.parameters(), lr=actor_lr)
optimizer_critic = optim.Adam(critic.parameters(), lr=critic_lr)

# making the environment
env = gym.make('LunarLander-v2')
rewards=[]

# Training loop
for episode in range(num_episodes):
    state = env.reset()
    net_reward = 0
    done=False
    for i in range(300):
        # Sample an action from the actor
        state_tensor = torch.tensor(state, dtype=torch.float32)
        action_probs = actor(state_tensor)
        action_distribution = torch.distributions.Categorical(action_probs)
        action = action_distribution.sample()
        next_state, reward, done, _ = env.step(action.item())

        # Calculating the TD error
        state_value = critic(state_tensor)
        next_state_value = critic(torch.tensor(next_state, dtype=torch.float32))
        td_error = reward + gamma * next_state_value - state_value

        # Updating the critic
        loss_critic = td_error.pow(2)
        optimizer_critic.zero_grad()
        loss_critic.backward()
        optimizer_critic.step()

        # Updating the actor using the TD error as a baseline
        log_prob = action_distribution.log_prob(action)
        loss_actor = -log_prob * td_error.detach()
        optimizer_actor.zero_grad()
        loss_actor.backward()
        optimizer_actor.step()

        net_reward += reward
        state = next_state
        if done== True:
          break


    print(f"Episode {episode + 1}/{num_episodes}, Net Reward: {net_reward}")

# Close the environment
env.close()


Episode 1/100, Net Reward: -411.49345164864155
Episode 2/100, Net Reward: -311.7580186458903
Episode 3/100, Net Reward: -373.7924111866449
Episode 4/100, Net Reward: -906.9574236441428
Episode 5/100, Net Reward: -501.1808343031242
Episode 6/100, Net Reward: -666.1993688693389
Episode 7/100, Net Reward: -337.40778379225765
Episode 8/100, Net Reward: -793.9006421259501
Episode 9/100, Net Reward: -709.7242550717442
Episode 10/100, Net Reward: -503.5136600800073
Episode 11/100, Net Reward: -706.213082570204
Episode 12/100, Net Reward: -671.1862284566914
Episode 13/100, Net Reward: -585.3914560761892
Episode 14/100, Net Reward: -666.7249006302175
Episode 15/100, Net Reward: -799.1695983854349
Episode 16/100, Net Reward: -568.7896248154786
Episode 17/100, Net Reward: -724.3310081898865
Episode 18/100, Net Reward: -813.9319642144839
Episode 19/100, Net Reward: -446.97495001615204
Episode 20/100, Net Reward: -388.50514772954864
Episode 21/100, Net Reward: -442.33105176852723
Episode 22/100, Ne